In [1]:
EXPORT_DIR = "../survey/import/"

In [2]:
from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
from explainer_wrappers import LIME_Explainer, SHAP_Explainer, Anchor_Explainer

In [3]:
explainer_classes = [LIME_Explainer, SHAP_Explainer]
detector_classes = [ DetectorGuo, DetectorRadford]

In [4]:
import pandas as pd
import json
import os

In [5]:
df = pd.read_pickle("./dataset_user_study_mock.pkl")


In [6]:
df

,Detector,Explainer,Documents Phases 1+3,Documents Phases 2+4,f(a),f(b),GT a,GT b,idx a,idx b,hash a,hash b
0,DetectorGuo,LIME_Explainer,The UK has historically aggressive financial l...,The word bespoke means made to order. Bespoke ...,1,1,True,True,237,276,8f1e6fbc370ec179204b1bddc3deb954e294f8e3c4e61c...,c4ca38facec5ee2dc55e7cfbf11c3df5415d01432efdc4...
1,DetectorGuo,LIME_Explainer,"I think the answer to this is just ""no."" It's...","Hi, one of the reasons for delay in period in ...",1,1,True,True,139,198,5c4703ed572fdad055906b8811355e7a3ad3794ecc2ed2...,3e70892a7fa30135b1f125feac4ee090dfa50d1a07f97a...
2,DetectorGuo,SHAP_Explainer,It is generally not possible for an adult to b...,Ex Machina is a 2014 science fiction film writ...,0,0,False,False,59,68,bf5cc4428ba8a20b7f40625ce7893a43aa9b3652f73d1c...,f5851c5a8dedccdfae60f3db0e8db835f5e567b6f8f88a...
3,DetectorGuo,SHAP_Explainer,Depends on how you measure liquidity. There's...,"The reason is, stores want customers to use ca...",1,1,True,True,30,121,48ccd95896b29bef9ffd5a14addf3235e368318b8a7f66...,1c3c9f892ad6b9339c6c9e647691a8fd193d5e7c537909...
4,DetectorGuo,Anchor_Explainer,"Yes, Volvo is a public company. It is listed o...",Chase credit cards are widely accepted for pur...,0,0,False,False,15,111,6bec45b04093791ca905d5a2b6a7fa6a905a64f965b546...,0d0911b39b0d882b399d09f73a5014af915ac9232e1594...
5,DetectorGuo,Anchor_Explainer,I've heard of handyman type people making a li...,Both are saying essentially the same thing. T...,1,1,True,True,0,52,6a66e7a115420485acd54c0cb9e5dc4e9e508b29af4b54...,714b04dd8923e09ea3f370b93660441d792104140d13d3...


In [7]:
if not os.path.exists(os.path.join(EXPORT_DIR)): 
    os.makedirs(os.path.join(EXPORT_DIR))
if not os.path.exists(os.path.join(EXPORT_DIR, "explanations")): 
    os.makedirs(os.path.join(EXPORT_DIR, "explanations"))
if not os.path.exists(os.path.join(EXPORT_DIR, "explanations", "html")): 
    os.makedirs(os.path.join(EXPORT_DIR, "explanations", "html"))
if not os.path.exists(os.path.join(EXPORT_DIR, "explanations", "data")): 
    os.makedirs(os.path.join(EXPORT_DIR, "explanations", "data"))

In [8]:
from IPython.core.display import display, HTML

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display


In [9]:

for detector_class in detector_classes:
    detector = detector_class()
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        for index, row in df.iterrows():
            a = row["Documents Phases 1+3"]
            b = row["Documents Phases 2+4"]
            
            explainer.get_fi_scores_batch([row["Documents Phases 1+3"],row["Documents Phases 2+4"] ]) # caches explanations
            # Phase 1 + 3: 
            path_explanation_html = os.path.join(EXPORT_DIR, "explanations", "html", explainer.get_hash(a)+".html")
            with open(path_explanation_html, "w", encoding="UTF-8") as text_file:
                text_file.write(explainer.get_highlighted_text_HTML(a))
            path_explanation_json = os.path.join(EXPORT_DIR, "explanations", "data", explainer.get_hash(a)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "ground_truth" : row["GT a"],
                    "detector_label" : int(detector.predict_label([row["Documents Phases 1+3"]])[0]),
                    "detector_p_machine": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][0]),
                    "detector_p_human": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][1]),
                    "document": row["Documents Phases 1+3"],
                    "explanation_filename": explainer.get_hash(a),               
                }
                text_file.write(json.dumps(explanation_data))
            # Phase 2+4: Just string
            path_explanation_html = os.path.join(EXPORT_DIR, "explanations", "html", explainer.get_hash(b)+".html")
           # display(HTML(explainer.get_highlighted_text_HTML(b)))
            with open(path_explanation_html, "w", encoding="UTF-8") as text_file:
                text_file.write(explainer.get_highlighted_text_HTML(b))
            path_explanation_json = os.path.join(EXPORT_DIR, "explanations", "data", explainer.get_hash(b)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "document": row["Documents Phases 2+4"]                
                }
                text_file.write(json.dumps(explanation_data))
                
            

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating explanations: 100%|██████████| 2/2 [00:00<00:00, 1992.07it/s]


In [10]:
[detector_class.__name__ for detector_class in detector_classes ]

['DetectorGuo', 'DetectorRadford']